In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Installing SimpleTransformers

In [ ]:
!pip install git+git://github.com/AndLen/simpletransformers.git --quiet

ERROR: Command errored out with exit status 128: git clone -q git://github.com/AndLen/simpletransformers.git /tmp/pip-req-build-9mqz3rk3 Check the logs for full command output.


In [ ]:
!pip install simpletransformers
!pip install transformers

     |████████████████████████████████| 248 kB 4.3 MB/s 
     |████████████████████████████████| 325 kB 48.3 MB/s 
     |████████████████████████████████| 1.2 MB 39.5 MB/s 
     |████████████████████████████████| 1.7 MB 31.9 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 9.9 MB 35.4 MB/s 
     |████████████████████████████████| 144 kB 52.4 MB/s 
     |████████████████████████████████| 181 kB 46.5 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 1.1 MB 42.3 MB/s 
     |████████████████████████████████| 134 kB 54.5 MB/s 
     |████████████████████████████████| 212 kB 53.1 MB/s 
     |████████████████████████████████| 127 kB 50.8 MB/s 
     |████████████████████████████████| 271 kB 53.3 MB/s 
     |████████████████████████████████| 94 kB 3.0 MB/s 
     |████████████████████████████████| 144 kB 33.8 MB/s 
     |████████████████████████████████| 4.3 MB 37.3 MB/s 
     |███████████████

In [ ]:
import csv
import os
import torch
from transformers import pipeline
import gc
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.special import softmax
from simpletransformers.classification import (ClassificationModel, ClassificationArgs)
import sklearn
from sklearn.model_selection import train_test_split

## Loading Data

In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google import colab
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Colab Notebooks/FYP

Mounted at /gdrive
/gdrive/My Drive/Colab Notebooks/FYP


In [ ]:
test = pd.read_csv("cyberbullying_tweets.csv", nrows=9538)
training = pd.read_csv("cyberbullying_tweets.csv")

In [ ]:
training["TWEET_TEXT"].isna().sum()

0

In [ ]:
training_df = training[["TWEET_TEXT", "RESULT"]]
training_df.columns = ["TEXT", "RES"]

In [ ]:
training_df.shape

(47692, 2)

In [ ]:
shuffled_training = training_df.sample(frac=1).reset_index(drop=True)

## Train, test, eval splitting with 70:15:15 proportions

Used for model selection, finally I train the classifier on the whole training set

In [ ]:
#train_df, test_df = train_test_split(training_df, test_size=0.15, random_state=42, stratify=training_df["labels"])
#eval_df, test_df = train_test_split(test_df, test_size=0.50, random_state=42, stratify=test_df["labels"])

In [ ]:
#train_df.shape ,test_df.shape, eval_df.shape

In [ ]:
# This cleans ram and vram during re-runs
gc.collect()
torch.cuda.empty_cache()

## Loading Bertweet from Huggingface

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
# Create a ClassificationModel
model_args = ClassificationArgs(num_train_epochs=2, 
                                overwrite_output_dir=True)
model_args.manual_seed = 42
model_args.best_model_dir = "/kaggle/working/best_model"
model_args.output_dir = "/kaggle/temp/output"
model_args.normalization = True #this enables the built-in Bertweet custom tokenizer

model_args.reprocess_input_data = True
#odel_args.evaluate_during_training = True
#model_args.evaluate_during_training_verbose = True
model_args.train_batch_size = 80
model_args.eval_batch_size = 80

model_args.early_stopping_metric = "mcc"
model_args.early_stopping_metric_minimize = False
model_args.use_early_stopping = True
model_args.early_stopping_consider_epochs = True
model_args.early_stopping_patience = 1

model = ClassificationModel(model_type='bertweet', 
                            model_name='vinai/bertweet-base', 
                            args = model_args, 
                            num_labels = 2)

Downloading:   0%|          | 0.00/558 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/517M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: 

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Please install emoji: pip3 install emoji
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model.train_model(shuffled_training,
                  acc=sklearn.metrics.accuracy_score, 
                  f1=sklearn.metrics.f1_score)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/47692 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/597 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/597 [00:00<?, ?it/s]

(1194, 0.22512158090045903)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(shuffled_training, 
                                                            acc=sklearn.metrics.accuracy_score,
                                                            f1=sklearn.metrics.f1_score)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/47692 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/597 [00:00<?, ?it/s]

In [ ]:
#0.85 accuracy on test set
result

{'acc': 0.930889876708882,
 'auprc': 0.9936483248832497,
 'auroc': 0.9687882385351033,
 'eval_loss': 0.1582453025620387,
 'f1': 0.9595215287500307,
 'fn': 682,
 'fp': 2614,
 'mcc': 0.7339380481847202,
 'tn': 5331,
 'tp': 39065}

## Predictions on new data

In [ ]:
predictions, raw_outputs = model.predict(test["TWEET_TEXT"].to_list())

  0%|          | 0/9538 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

In [ ]:
mypreds = pd.DataFrame(test[["RESULT"]])
mypreds["target"] = predictions

In [ ]:
mypreds.to_csv("submission.csv", index=False)